##Install Library

In [1]:
!pip install streamlit
!pip install streamlit-authenticator
!pip install whisperx
!pip install torch
!pip install pyyaml
!pip install PyPDF2
!pip install pyngrok


##Script Database

In [1]:
%%writefile database.py
import sqlite3

def init_db():
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE,
            password TEXT,
            email TEXT UNIQUE,
            role TEXT
        )
    ''')
    conn.commit()
    conn.close()

def create_user(username, password, email, role='user'):
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('INSERT INTO users (username, password, email, role) VALUES (?, ?, ?, ?)',
              (username, password, email, role))
    conn.commit()
    conn.close()

def get_user(username):
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('SELECT * FROM users WHERE username = ?', (username,))
    user = c.fetchone()
    conn.close()
    return user

def update_user_password(username, new_password):
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('UPDATE users SET password = ? WHERE username = ?', (new_password, username))
    conn.commit()
    conn.close()

def get_all_users():
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('SELECT * FROM users')
    users = c.fetchall()
    conn.close()
    return users


Overwriting database.py


## Script Authentification

In [2]:
%%writefile auth.py
import bcrypt
from database import create_user, get_user, update_user_password

def hash_password(password):
    """Hash the password with bcrypt."""
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

def check_password(password, hashed_password):
    """Check if the provided password matches the hashed password."""
    if isinstance(hashed_password, str):
        hashed_password = hashed_password.encode('utf-8')
    return bcrypt.checkpw(password.encode('utf-8'), hashed_password)

def register_user(username, password, email):
    """Register a new user with a hashed password."""
    if get_user(username):
        raise ValueError("Username already exists")
    hashed_password = hash_password(password)
    create_user(username, hashed_password.decode('utf-8'), email)

def authenticate_user(username, password):
    """Authenticate a user by checking the provided password against the stored hash."""
    user = get_user(username)
    if user:
        stored_password = user[2]
        if check_password(password, stored_password):
            return True
    return False

def reset_password(username, new_password):
    """Reset the password for the specified user."""
    hashed_password = hash_password(new_password)
    update_user_password(username, hashed_password.decode('utf-8'))


Overwriting auth.py


## Script Transcription

In [3]:
%%writefile transcription.py
import streamlit as st
import whisperx
import torch
import os


device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 16
compute_type = "float16" if torch.cuda.is_available() else "int8"

def extract_text_from_audio():
    st.title("Transcription Audio avec WhisperX")


    uploaded_file = st.file_uploader("Téléchargez un fichier audio (mp3 ou wav)", type=["mp3", "wav"])

    if uploaded_file:

        temp_file_path = os.path.join("/tmp", uploaded_file.name)
        with open(temp_file_path, "wb") as f:
            f.write(uploaded_file.getvalue())


        st.audio(temp_file_path, format='audio/mp3')


        model = whisperx.load_model("large-v2", device, compute_type=compute_type)


        audio = whisperx.load_audio(temp_file_path)


        st.write("Transcription en cours...")
        result = model.transcribe(audio, batch_size=batch_size)


        st.write("Premiers segments avant alignement :")
        for segment in result["segments"][:5]:
            st.write(segment)


        st.write("Alignement en cours...")
        model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
        result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)


        st.write("Premiers segments après alignement :")
        for segment in result["segments"][:5]:
            st.write(segment)


        transcription_text = "\n".join([segment["text"] for segment in result["segments"]])


        st.subheader("Transcription complète :")
        st.text_area("Transcription", transcription_text, height=250)


        st.download_button(
            label="Télécharger la transcription",
            data=transcription_text,
            file_name="transcription.txt",
            mime="text/plain"
        )


        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)


Overwriting transcription.py


## Script Admin

In [4]:
%%writefile admin.py
import streamlit as st
from database import get_all_users, update_user_password, get_user
import bcrypt

def admin_interface():
    st.title("Admin Interface")


    if st.session_state.username != 'admin':
        st.error("You do not have permission to access this page.")
        return

    st.subheader("User Management")

    users = get_all_users()
    if users:
        for user in users:
            st.write(f"Username: {user[1]}, Email: {user[3]}, Role: {user[4]}")
            if st.button(f"Reset Password for {user[1]}"):
                new_password = st.text_input(f"New password for {user[1]}", type="password")
                if st.button(f"Confirm Reset for {user[1]}"):
                    update_user_password(user[1], bcrypt.hashpw(new_password.encode('utf-8'), bcrypt.gensalt()).decode('utf-8'))
                    st.success(f"Password for {user[1]} has been reset.")
            st.write("---")
    else:
        st.write("No users found.")


Overwriting admin.py


## Script NFS

In [5]:
%%writefile nfs.py
import streamlit as st
import PyPDF2
import re
import pandas as pd


def extract_text_from_pdf(pdf_file):
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text


def clean_text(text):
    text = text.replace("Ã‰", "É").replace("â", " ").replace("", "⁶").replace("Âµ", "µ").replace("Â³", "³").replace("fl", "fL")
    return text


def extract_fields(text):
    patterns = {
        'nom_patient': r'Nom\s*:\s*(.*?)\n',
        'medecin_traitant': r'Médecin\s*:\s*(.*?)\n',
        'HÉMATIES': r'HÉMATIES\s*(\d+\.?\d*)\s*(10⁶/µL)\s*\((.*?)\)',
        'HÉMATOCRITE': r'HÉMATOCRITE\s*(\d+\.?\d*)\s*(%)\s*\((.*?)\)',
        'HÉMOGLOBINE': r'HÉMOGLOBINE\s*(\d+\.?\d*)\s*(g/dL)\s*\((.*?)\)',
        'V.G.M.': r'V.G.M.\s*(\d+\.?\d*)\s*(fL)\s*\((.*?)\)',
        'C.C.M.H.': r'C.C.M.H.\s*(\d+\.?\d*)\s*(g/dL)\s*\((.*?)\)',
        'T.C.M.H.': r'T.C.M.H.\s*(\d+\.?\d*)\s*(pg)\s*\((.*?)\)',
        'RDW': r'RDW\s*(\d+\.?\d*)\s*(%)\s*\((.*?)\)',
        'LEUCOCYTES': r'LEUCOCYTES\s*(\d+\.?\d*)\s*(/mm3)\s*\((.*?)\)',
        'Polynucléaires Neutrophiles': r'Polynucléaires Neutrophiles\s*(\d+\.?\d*)\s*(%)\s*Soit\s*:\s*(\d+\.?\d*)\s*(/mm3)\s*\((.*?)\)',
        'Polynucléaires Eosinophiles': r'Polynucléaires Eosinophiles\s*(\d+\.?\d*)\s*(%)\s*Soit\s*:\s*(\d+\.?\d*)\s*(/mm3)\s*\((.*?)\)',
        'Polynucléaires Basophiles': r'Polynucléaires Basophiles\s*(\d+\.?\d*)\s*(%)\s*Soit\s*:\s*(\d+\.?\d*)\s*(/mm3)\s*\((.*?)\)',
        'Lymphocytes': r'Lymphocytes\s*(\d+\.?\d*)\s*(%)\s*Soit\s*:\s*(\d+\.?\d*)\s*(/mm3)\s*\((.*?)\)',
        'Monocytes': r'Monocytes\s*(\d+\.?\d*)\s*(%)\s*Soit\s*:\s*(\d+\.?\d*)\s*(/mm3)\s*\((.*?)\)',
        'NUMERATION DES PLAQUETTES': r'NUMERATION DES PLAQUETTES\s*(\d+\.?\d*)\s*(10³/mm3)\s*\((.*?)\)',
        'Volume plaquettaire moyen': r'Volume plaquettaire moyen\s*(\d+\.?\d*)\s*(fL)\s*\((.*?)\)',
    }

    extracted_fields = {}
    for field, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            extracted_fields[field] = ' '.join(match.groups())

    return extracted_fields


def analyze_fields_with_reference(df):
    analysis = []
    for _, row in df.iterrows():
        field = row['Champ']
        value = row['Valeur']
        ref = row['Référence']


        try:
            value_num = float(re.findall(r"[\d.]+", value)[0])
        except IndexError:
            value_num = None

        try:
            ref_range = re.findall(r"[\d.]+", ref)
            ref_min = float(ref_range[0])
            ref_max = float(ref_range[1])
        except (IndexError, ValueError):
            ref_min = ref_max = None


        if value_num is not None and ref_min is not None and ref_max is not None:
            if value_num < ref_min:
                status = "Below normal"
            elif value_num > ref_max:
                status = "Above normal"
            else:
                status = "Normal"
        else:
            status = "Unable to determine"

        analysis.append([field, value, status, ref])

    return pd.DataFrame(analysis, columns=["Champ", "Valeur", "État", "Plage de Référence"])


def generate_health_summary(df):
    general_status = {
        "Normal": 0,
        "Below normal": 0,
        "Above normal": 0
    }

    for _, row in df.iterrows():
        status = row['État']
        if status in general_status:
            general_status[status] += 1

    total_fields = len(df)
    summary = {
        "Normal": f"{general_status['Normal']} / {total_fields}",
        "Below normal": f"{general_status['Below normal']} / {total_fields}",
        "Above normal": f"{general_status['Above normal']} / {total_fields}"
    }

    if general_status['Above normal'] > general_status['Normal']:
        overall_status = "Potential health issues detected."
    elif general_status['Below normal'] > general_status['Normal']:
        overall_status = "Potential deficiencies detected."
    else:
        overall_status = "Blood test results are within normal range."

    return summary, overall_status


def generate_health_advice(df):
    advice = []

    for _, row in df.iterrows():
        field = row['Champ']
        status = row['État']

        if status == "Below normal":
            if field in ['HÉMATIES', 'HÉMATOCRITE', 'HÉMOGLOBINE']:
                advice.append(f"Low {field}: Consider increasing intake of iron-rich foods such as red meat, legumes, and dark leafy greens. Consult a healthcare provider if symptoms persist.")
            elif field in ['LEUCOCYTES']:
                advice.append(f"Low {field}: This could indicate an issue with the immune system. Consult a healthcare provider for further evaluation.")
            elif field in ['NUMERATION DES PLAQUETTES']:
                advice.append(f"Low {field}: This may require medical evaluation to rule out underlying conditions.")
            else:
                advice.append(f"Low {field}: Consult with a healthcare provider for personalized advice.")

        elif status == "Above normal":
            if field in ['HÉMATIES', 'HÉMATOCRITE', 'HÉMOGLOBINE']:
                advice.append(f"High {field}: This may indicate dehydration or other health conditions. Increase fluid intake and consult with a healthcare provider.")
            elif field in ['LEUCOCYTES']:
                advice.append(f"High {field}: This may suggest an infection or inflammation. Consult a healthcare provider for a proper diagnosis.")
            elif field in ['NUMERATION DES PLAQUETTES']:
                advice.append(f"High {field}: This may be related to an inflammatory or clotting disorder. Consult a healthcare provider.")
            else:
                advice.append(f"High {field}: Consult with a healthcare provider for personalized advice.")
        else:
            advice.append(f"{field}: Results are within normal range. No specific advice required.")

    return advice


def process_pdf(uploaded_file):
    if uploaded_file is not None:

        pdf_text = extract_text_from_pdf(uploaded_file)


        pdf_text = clean_text(pdf_text)


        extracted_fields = extract_fields(pdf_text)


        patient_doctor_info = {
            'nom_patient': extracted_fields.pop('nom_patient', ''),
            'medecin_traitant': extracted_fields.pop('medecin_traitant', '')
        }


        df_patient_doctor = pd.DataFrame(list(patient_doctor_info.items()), columns=["Champ", "Valeur"])


        other_fields = {k: v for k, v in extracted_fields.items() if v}
        df_other = pd.DataFrame([
            [k] + re.split(r'\s+', v, 1)
            for k, v in other_fields.items()
        ], columns=["Champ", "Valeur", "Référence"])


        analysis_df = analyze_fields_with_reference(df_other)


        summary, overall_status = generate_health_summary(analysis_df)
        health_advice = generate_health_advice(analysis_df)

        return df_patient_doctor, analysis_df, summary, overall_status, health_advice


def main():
    st.title("PDF Blood Test Extractor")
    st.write("Upload a PDF document to extract blood test results and analyze the data.")


    uploaded_file = st.file_uploader("Upload PDF", type="pdf")

    if uploaded_file:
        df_patient_doctor, analysis_df, summary, overall_status, health_advice = process_pdf(uploaded_file)

        st.header("Patient and Doctor Information")
        st.dataframe(df_patient_doctor)

        st.header("Blood Test Results Analysis")
        st.dataframe(analysis_df)

        st.header("Health Summary")
        st.write(f"Normal: {summary['Normal']}")
        st.write(f"Below Normal: {summary['Below normal']}")
        st.write(f"Above Normal: {summary['Above normal']}")
        st.write(f"Overall Status: {overall_status}")

        st.header("Health Advice")
        for advice in health_advice:
            st.write(f"- {advice}")

if __name__ == "__main__":
    main()


Overwriting nfs.py


## Script Main

In [6]:
%%writefile main.py
import streamlit as st
from database import init_db, get_user, get_all_users
from auth import register_user, authenticate_user, reset_password
from transcription import extract_text_from_audio
from nfs import extract_text_from_pdf, clean_text, extract_fields, process_pdf


init_db()

st.sidebar.image('/content/logo.png', use_column_width=False, width=150)

def main():

    st.markdown("""
      <style>
      .main {
          background-color: #333333; /* Charcoal background color */
          color: #ffffff; /* White text color for better readability */
      }
      .sidebar .sidebar-content {
          background-color: #2F4F4F; /* Dark Slate Gray for sidebar */
          color: #ffffff; /* White text color */
      }
      .stButton>button {
          background-color: #353839; /* Onyx for buttons */
          color: white;
      }
      .stTitle {
          text-align: center; /* Center align title */
      }
      .image-container {
          display: flex;
          justify-content: center;
          gap: 20px;
          margin-top: 20px;
      }
      .image-container img {
          width: 300px;
          height: auto;
          border-radius: 8px;
      }
      .logo {
          display: flex;
          justify-content: center;
          margin-bottom: 20px;
      }
      .logo img {
          width: 150px;
          height: auto;
      }
      </style>
  """, unsafe_allow_html=True)




    if 'authenticated' not in st.session_state:
        st.session_state.authenticated = False

    if 'username' not in st.session_state:
        st.session_state.username = None


    st.sidebar.title("Navigation")


    if st.session_state.authenticated:
        options = ["Accueil", "Transcription Audio", "Extraction de Tableaux NFS", "Admin", "Déconnexion"]
    else:
        options = ["Accueil", "Se Connecter", "S'inscrire", "Mot de Passe Oublié"]

    option = st.sidebar.selectbox("Choisissez une page", options)

    if option == "Accueil":
        st.markdown("""
            <h1 class="stTitle">Bienvenue dans l'application d'analyse médicale</h1>
            <p style="text-align: center; color: #00796b;">Utilisez la barre latérale pour naviguer entre les différentes fonctionnalités.</p>
        """, unsafe_allow_html=True)
        st.image("/content/nfs.jpg", use_column_width=True, caption="")
        st.image("/content/download.jpeg", use_column_width=True, caption="")

    elif option == "Se Connecter":
        st.title("Se Connecter")
        username = st.text_input("Nom d'utilisateur")
        password = st.text_input("Mot de passe", type="password")
        if st.button("Se Connecter"):
            if authenticate_user(username, password):
                st.session_state.authenticated = True
                st.session_state.username = username
                st.success("Connexion réussie!")
            else:
                st.error("Nom d'utilisateur ou mot de passe incorrect.")

    elif option == "S'inscrire":
        st.title("S'inscrire")
        username = st.text_input("Nom d'utilisateur")
        password = st.text_input("Mot de passe", type="password")
        email = st.text_input("Email")
        if st.button("S'inscrire"):
            try:
                register_user(username, password, email)
                st.success("Inscription réussie! Vous pouvez maintenant vous connecter.")
            except ValueError as e:
                st.error(str(e))

    elif option == "Mot de Passe Oublié":
        st.title("Réinitialiser le mot de passe")
        username = st.text_input("Nom d'utilisateur")
        new_password = st.text_input("Nouveau mot de passe", type="password")
        if st.button("Réinitialiser le mot de passe"):
            try:
                reset_password(username, new_password)
                st.success("Mot de passe réinitialisé avec succès!")
            except Exception as e:
                st.error("Erreur lors de la réinitialisation du mot de passe.")

    elif option == "Transcription Audio":
        extract_text_from_audio()

    elif option == "Extraction de Tableaux NFS":
        st.title("Extraction de Tableaux NFS")
        st.write("Téléchargez un fichier PDF pour extraire les résultats des tests sanguins et analyser les données.")
        uploaded_file = st.file_uploader("Téléchargez un fichier PDF", type=["pdf"])

        if uploaded_file:
            try:
                df_patient_doctor, df_analysis_with_reference, summary, overall_status, health_advice = process_pdf(uploaded_file)

                st.header("Informations sur le Patient et le Médecin")
                st.dataframe(df_patient_doctor)

                st.header("Analyse des Résultats des Tests Sanguins")
                st.dataframe(df_analysis_with_reference)

                st.header("Résumé de la Santé")
                st.write(f"Normal: {summary['Normal']}")
                st.write(f"En-dessous de la normale: {summary['Below normal']}")
                st.write(f"Au-dessus de la normale: {summary['Above normal']}")
                st.write(f"État global: {overall_status}")

                st.header("Conseils de Santé")
                for advice in health_advice:
                    st.write(f"- {advice}")

            except KeyError as e:
                st.error(f"Erreur de données: {str(e)}")
            except Exception as e:
                st.error(f"Erreur inattendue: {str(e)}")

    elif option == "Admin" and st.session_state.authenticated:
        st.title("Admin")
        st.write("Liste des utilisateurs inscrits:")
        users = get_all_users()
        st.write(users)

    elif option == "Déconnexion":
        st.session_state.authenticated = False
        st.session_state.username = None
        st.success("Déconnecté avec succès.")

if __name__ == "__main__":
    main()


Overwriting main.py


## Connection to NGROK

In [9]:
import os
from pyngrok import ngrok
ngrok.set_auth_token('2l3ZMJuiCVuO8fONnKFWe1uXW9x_3VdvGuu9JchHseJ7oL8zi')

In [10]:
!streamlit run  main.py &>/dev/null &
from pyngrok import ngrok
public_url = ngrok.connect(addr="8502", proto="http")
print(f'Application publique URL: {public_url}')


Application publique URL: NgrokTunnel: "https://e970-35-227-96-102.ngrok-free.app" -> "http://localhost:8502"
